# 0. Library

In [22]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import re, time, copy
from tqdm import tqdm
from itertools import chain
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100% !important;}</style>"))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.nn import functional
from torch.utils.data import DataLoader, Dataset

import transformers
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

# 1. Config

In [23]:
class Config:
    bos_token = '<s>'
    eos_token = '</s>'
    usr_token = '<usr>'
    pad_token = '<pad>'
    sys_token = '<sys>'
    unk_token = '<unk>'
    mask_token = '<mask>'
    max_length = 384
    max_turns = 6
    epochs = 4
    batch_size = 8
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    learning_rate = 1e-4
    model_name = "skt/kogpt2-base-v2"

# 2. Data

In [24]:
N = 1000
train_per = 0.95

data = pd.read_csv('./data/kakao_preprocess.csv')
data_train_val = data.sample(N).reset_index(drop=True)
data_train_val = data.reset_index(drop=True)
data_train = data_train_val[: int(len(data_train_val) * train_per)]
data_val = data_train_val[int(len(data_train_val) * train_per): ].reset_index(drop=True)
data_train

,conversation
0,<usr> 나 운전면허 따야 하는데<sys> 너 따긴 했잖아?연수를 받아야지 너는<...
1,<usr> 내일은 산에 못가시나요?<sys> 산에 못가고 장보러 가야지.휴양림가서 ...
2,<usr> 오늘 도서관가서 책 빌려왔음<sys> 오호 도서관도 가나? 어디도서관?<...
3,<usr> 파운데이션 하나 같이 테스트 하러 가자<sys> 오 파운데이션 다 썼어?...
4,<usr> 비긴 어게인 다시 보고 싶다<sys> 아 나는 그거 또보고싶다고!<usr...
...,...
59523,<usr> 첫째랑 둘째 공주 놀이한다고 ㅋㅋ나 티비 보라는데 ㅋㅋ 볼게없는..ㅠ<s...
59524,<usr> 우리도 캠핑 가보자<sys> 옥상테라스에 텐트치면 되지 뭘 어딜가.<us...
59525,<usr> 너는 군대간 사람 기다릴 수 있을 거 같아?<sys> 글쎄. 누구냐에 따...
59526,<usr> 나 요즘 좀 우울한 것 같아<sys> 엥? 갑자기 왜? 무슨 일 있어?<...


In [25]:
data_val

,conversation
0,<usr> 원트 메가 크루 미션 봤어?<sys> 웅웅 되게 파이팅 넘치던데?<us...
1,<usr> 자전거를 타고 출퇴근 해볼까<sys> 안양에서 서울까지 자전거로?<usr...
2,<usr> 연애할 때 가까워 지기 위해서는 상대방 무엇을 알아야할까?<sys> 상대...
3,<usr> 10월 25일이 독도의 날이란다<sys> 독도의 날? 그런게 있었어?<u...
4,<usr> 면허를 따도 쓸 일이 없네<sys> 장롱면허인지 10년 째<usr> 차를...
...,...
3129,<usr> 나 방금 병원 다녀왔어 ㅠㅠ<sys> 병원? 어디 아파ㅠ?<usr> 목이...
3130,<usr> 영화볼 때 주인공 위주로 보는 편이야?<sys> 응 난 아무래도 주인공이...
3131,<usr> 포도가 요즘 맛있더라고요.<sys> 무슨 포도 드셨어요?<usr> 샤인머...
3132,<usr> 오늘 문화센터 특강한던데 한번 가볼래?<sys> 그거 등록해야하는거 아니...


# 3. Tokenizer

In [26]:
tokenizer = PreTrainedTokenizerFast.from_pretrained(Config.model_name,
            bos_token=Config.bos_token, eos_token=Config.eos_token,
            unk_token=Config.unk_token, pad_token=Config.pad_token,
            mask_token=Config.mask_token, model_max_length=Config.max_length)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [27]:
for i in range(10):
    print(tokenizer.convert_ids_to_tokens(i), end=' ')

<s> </s> <usr> <pad> <sys> <unk> <mask> <d> </d> <unused0> 

# 4. Dataloader

In [31]:
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, Config):
        self.data = data
        self.tokenizer = tokenizer
        self.usr_token = Config.usr_token
        self.sys_token = Config.sys_token
        self.bos_token = Config.bos_token
        self.eos_token = Config.eos_token
        self.mask_token = Config.mask_token
        self.pad_token = Config.pad_token
        self.max_length = Config.max_length
        self.max_turns = Config.max_turns
        
    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self.data)
        
    def __getitem__(self, idx):
        sentence = self.data['conversation'][idx]
        # input_id
        input_id = self.tokenizer.encode(self.bos_token + sentence + self.eos_token)
        # token_type_id
        token_type = []
        loop = True
        for token_id in input_id:
            token = self.tokenizer.convert_ids_to_tokens(token_id)
            
            if token == self.usr_token: loop=True
            elif token == self.sys_token: loop=False
                
            if loop:
                token_type.append(self.usr_token)
            else:
                token_type.append(self.sys_token)
        token_type_id = self.tokenizer.convert_tokens_to_ids(token_type)
        # label
        start_idx = len(input_id) - list(reversed(input_id)).index(4)
        label = [-100] * start_idx + input_id[start_idx: ]
        # padding
        input_id, token_type_id, label = self.make_padding(input_id, token_type_id, label)
        
        return input_id, token_type_id, label

    def make_padding(self, input_id, token_type_id, label):
        left_length = self.max_length - len(input_id)
        input_id += [self.tokenizer.pad_token_id] * left_length
        token_type_id += [self.tokenizer.pad_token_id] * left_length
        label += [-100] * left_length
        
        return input_id, token_type_id, label

In [32]:
train_set = CustomDataset(data_train, tokenizer, Config)

In [33]:
input_id, token_type_id, label = train_set[0]
print("input_id", input_id, sep='\n')
print("token_type_id", token_type_id, sep='\n')
print("label", label, sep='\n')

input_id
[0, 2, 9063, 17970, 36100, 9106, 7991, 14696, 4, 10099, 9106, 6960, 9651, 8162, 7965, 406, 8033, 10007, 18408, 8263, 10099, 7162, 2, 15247, 9081, 10007, 13892, 18408, 9337, 9031, 27511, 389, 4, 9774, 9515, 29247, 14807, 2, 9716, 26861, 9685, 8263, 46651, 4, 9273, 11865, 9078, 7182, 18381, 9285, 7607, 12249, 27076, 2, 37472, 18882, 9098, 7661, 7991, 6872, 7098, 4, 10723, 17970, 9033, 6866, 9266, 9328, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [34]:
def collate_batch(batch):
    input_ids = [items[0] for items in batch]
    token_type_ids = [items[1] for items in batch]
    labels = [items[2] for items in batch]
    
    return torch.LongTensor(input_ids), torch.LongTensor(token_type_ids), \
            torch.LongTensor(labels)

In [35]:
train_dataloader = DataLoader(train_set, batch_size=Config.batch_size, num_workers=2,
                              shuffle=True, collate_fn=collate_batch)


In [36]:
val_set = CustomDataset(data_val, tokenizer, Config)
val_dataloader = DataLoader(val_set, batch_size=Config.batch_size, num_workers=2,
                            shuffle=False, collate_fn=collate_batch)


# 5. Model

In [37]:
model = GPT2LMHeadModel.from_pretrained(Config.model_name).to(Config.device)

# 6. Train

In [38]:
class ChatBot:
    def __init__(self, model, tokenizer, Config):
        self.model = model
        self.tokenizer = tokenizer
        self.optimizer = torch.optim.AdamW(model.parameters(), lr=Config.learning_rate)
        self.scheduler = torch.optim.lr_scheduler.ExponentialLR(self.optimizer, gamma=0.9)
        self.usr_token_id = tokenizer.get_vocab()[Config.usr_token]
        self.sys_token_id = tokenizer.get_vocab()[Config.sys_token]
        self.max_length = Config.max_length
        self.max_turns = Config.max_turns
        
        self.losses = []
        self.val_losses = []

    
    def train(self, epochs, train_dataloader, validation_dataloader=None, save=None):
        self.model.train()
        
        for epoch in range(epochs):
            print(f"\n Epoch {epoch+1}/{epochs}", sep="\n")
            start_time = time.time()
            batch_loss = []

            for i, batch in enumerate(train_dataloader):
                input_ids, token_type_ids, labels = batch        
                input_ids, token_type_ids, labels = input_ids.to(Config.device), token_type_ids.to(Config.device),\
                                                    labels.to(Config.device)
                outputs = self.model(
                    input_ids = input_ids,
                    token_type_ids = token_type_ids,
                    labels = labels
                )
                
                loss = outputs.loss
                
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                batch_loss.append(loss.item())
                
                print(self.status(i+1, len(train_dataloader), time.time()-start_time, np.mean(batch_loss)), end='\r')
            
            self.scheduler.step()
            
            self.losses.append(np.mean(batch_loss))
            
            if validation_dataloader:
                val_loss = self.validation(validation_dataloader)
                print(self.status(i+1, len(train_dataloader), time.time()-start_time, np.mean(batch_loss)) + \
                      " | val_loss : %.6f"%(val_loss), end='\r')
                self.val_losses.append(val_loss)
            
            if save:
                time_zone = datetime.timezone(datetime.timedelta(hours=9))
                now = datetime.datetime.now(time_zone)
                PATH = now.strftime(f'./check_point/%Y-%m-%d-%Hh-%Mm_epoch_{epoch+1+Config.pre_epochs}_sk_labeling_{Config.labeling_type}.pth')
                torch.save(self.model.state_dict(), PATH)

    def validation(self, validation_dataloader):
        self.model.eval()
        self.model.to(Config.device)
        batch_loss = []
        
        with torch.no_grad():
            for i, batch in enumerate(validation_dataloader):
                input_ids, token_type_ids, labels = batch
                input_ids, token_type_ids, labels = input_ids.to(Config.device), token_type_ids.to(Config.device),\
                                                    labels.to(Config.device)
                
                outputs = self.model(
                    input_ids = input_ids,
                    token_type_ids = token_type_ids,
                    labels = labels
                )
                
                loss = outputs.loss
                batch_loss.append(loss.item())
            
            valid_loss = np.mean(batch_loss)
        
        return valid_loss
    
    @staticmethod
    def status(step, step_len, time, loss):
        return f"step : {step}/{step_len} - {int(time)}s | loss : {loss:.6f} | {step/time:.2f}it/s"

    def save_model(self, PATH=None):
        if not PATH:
            now = datetime.datetime.now()
            now_date = now.strftime('%m%d_%H%M')
            PATH = 'models/' + str(now_date) + '_model.pt'
        torch.save(self.model.state_dict(), PATH)
        print("model saved.")

    def load_model(self, PATH):
        self.model.load_state_dict(torch.load(PATH))
        print("model loaded.")


In [39]:
chathuman = ChatBot(model, tokenizer, Config)

In [ ]:
chathuman.train(Config.epochs, train_dataloader, val_dataloader)


 Epoch 1/4


# 7. Inference

In [ ]:
PATH = 'models/model.pt'

In [ ]:
chathuman.save_model(PATH)

In [ ]:
chathuman = ChatBot(model, tokenizer, Config)

In [ ]:
chathuman.load_model(PATH)

In [ ]:
history = ""
start=True

while True:
    _input = input("user > ")
    
    if _input == "끝":
        break
    if _input == "초기화":
        history = ""
        continue
    if start:
        _input_word = tokenizer.bos_token + '<usr>' + _input + '<sys>'
        history += _input_word
        start = False
    else:
        _input_word = '<usr>' + _input + '<sys>'
        if sum([len(i) for i in history]) + len(_input_word) > 100: history = history[1:]
        history += _input_word
        
    input_ids = tokenizer.encode(history, return_tensors="pt").to(Config.device)
    
    with torch.no_grad():
        gen_ids = model.generate(
            input_ids,
            max_length=200,
            top_k=3,
            top_p=0.92,
            num_beams=7,
            do_samples=True,
            no_repeat_ngram_size=3,
            repetition_penalty=1,
            temperature=0.4,
            max_new_tokens=30,
            eos_token_id=tokenizer.eos_token_id
        )
    
    gen = tokenizer.decode(gen_ids[0])
    try:
        generated = gen[gen.rfind("<sys>")+5:gen.index("</s>")]
    except:
        generated = gen[gen.rfind("<sys>")+5:]
    history += generated
    
    print(f'Chatbot > {generated}')